In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

from IPython.display import clear_output
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
!pip install autoviz
!pip install xlrd

from autoviz.AutoViz_Class import AutoViz_Class
AV = AutoViz_Class()

clear_output()

# AUTO EDA

In [ ]:
ff = AV.AutoViz('../input/30-days-of-ml/train.csv')

# MODELING

In [ ]:
from xgboost import XGBRegressor
from xgboost import plot_tree
xgb_params = {
    'n_estimators': 5000,
    'learning_rate': 0.1235,
    'subsample': 0.95,
    'colsample_bytree': 0.11,
    'max_depth': 2,
    'booster': 'gbtree', 
    'reg_lambda': 66.1,
    'reg_alpha': 15.9,
    'random_state':42
}


In [ ]:
df = pd.read_csv("/kaggle/input/ten-folds/10_folds (1).csv")
df_test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

# KFOLDS

This dataset is the same as abishek thakur created!

In [ ]:
final_predictions = []
for fold in range(10):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    model = XGBRegressor(**xgb_params)
    model.fit(xtrain, ytrain)
    
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    print(f"fold: {fold}, rmse: {mean_squared_error(yvalid, preds_valid, squared=False)}")

# PREDICTIONS

In [ ]:
preds = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.target = preds
sample_submission.to_csv("xgb_submission_z", index=False)